# Supervised Learning with **PYTHON**

**INTRODUCTION**

In this practical, we introduce different machine learning algorithms to solve a classification problem. You will create a model for a hospital, to predict if a patient has breast cancer or not 



In this practical, we tackle the task of classification. Classification in machine learning involves learning a labelling of examples into one (or more) discrete categories. This differs from another common task in machine learning called regression, which involves learning a mapping from inputs to a continuous-valued output.

#### **Examples of Classification Problems**

Determine whether an email message (input) is SPAM or NOT SPAM (label)

Determine whether an image, represented by its encoded pixel values (input) is a picture of a DOG or a CAT (label)

About the Dataset 

The tagged data set is from the "Breast Cancer Wisconsin (Diagnostic) Database" freely available in python's sklearn library, for details see:
https://archive.ics.uci.edu/ml/datasets/Breast+Cancer+Wisconsin+%28Diagnostic%29

    Number of Samples: 569
    Number of Features: 30 numeric, predictive attributes
    Number of Classes: 2

Features are computed from a digitized image of a fine needle aspirate (FNA) of a breast mass. They describe characteristics of the cell nuclei present in the image. Ten real-valued features are computed for each cell nucleus. The mean, standard error and 'worst' or largest (mean of the three largest values) of these features were computed for each image, resulting in 30 features. For instance, the radius measurements are for the 'mean radius', 'standard error of the radius', and 'worst radius'. All feature values are recoded with four significant digits.

The two target classes correspond to negative outcomes (Benign) and positive outcomes (Malignant).

---



Importing packages

In [0]:
import pandas as pd # for data manipulation
import numpy as np # for mathematical operations
import matplotlib.pyplot as plt # for plotting and visualizations
import seaborn as sns # for plotting and visualizations

# algorithms
from sklearn.datasets import load_breast_cancer # for loading the inbuilt dataset
from sklearn.linear_model import LogisticRegression as LR # for logistic regression
from sklearn.ensemble import RandomForestClassifier as RF # for random forest algorithm
from sklearn.ensemble import GradientBoostingClassifier as GBC # for gradient boost
from sklearn.naive_bayes import GaussianNB as GNB # for gaussian naive bayes
from sklearn import svm # for support vector machine

from sklearn.model_selection import train_test_split # for split the data into train and test

# Metrics
from sklearn.metrics import confusion_matrix # for checking the confusion matrix
from sklearn.metrics import precision_recall_curve # for checking the precision and recall curves
from sklearn.metrics import roc_curve, auc # for the area under curve and roc metrics

Loading in the data using built in dataset from sklearn

In [0]:
# df = pd.read_csv('/path/to/file.csv')


In [0]:
# Load the dataset
Data = load_breast_cancer()

In [0]:
# print a description of the dataset
print(Data.DESCR)

In [0]:
# View the first 5 records of the dataset features
df = pd.DataFrame(Data.data, columns = Data.feature_names)
df.head()

In [0]:
#Describe the numerical properties of the data
df.describe()

In [0]:
# df.shape
df.shape

In [0]:
# information
df.info()

In [0]:
# missing values
df.isnull().sum()


### **Describe the Feature Statistics**

In [0]:
print("The sklearn breast cancer dataset keys:")
print(Data.keys()) # dict_keys(['target_names', 'target', 'feature_names', 'data', 'DESCR'])
print("---")

# Note that we need to reverse the original '0' and '1' mapping in order to end up with this mapping:
# Benign = 0 (negative class)
# Malignant = 1 (positive class)

li_classes = [Data.target_names[1], Data.target_names[0]]
li_target = [1 if x==0 else 0 for x in list(Data.target)]
li_ftrs = list(Data.feature_names)

print("There are 2 target classes:")
print("li_classes", li_classes)
print("---")
print("Target class distribution from a total of %d target values:" % len(li_target))
print(pd.Series(li_target).value_counts())
print("---")

df_all = pd.DataFrame(Data.data[:,:], columns=li_ftrs)
print("Describe dataframe, first 6 columns:")
print(df_all.iloc[:,:6].describe().to_string())

### **TRAIN/TEST SPLITTING**

Split the data into train and test sets

In [0]:
TEST_SIZE_RATIO = 0.2   # the training set will be 80% the test set will be 20%
# Setup X and y
X = df_all
y = pd.Series(li_target)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=TEST_SIZE_RATIO, random_state=0)

The labels are y and features are X, since sklearn.model_selection.train_test_split does a random split every time it is called, specifying a random_state value ensures that the split is the same. The split ratio in this case is train:test=80:20 

In [0]:
yX = pd.concat([y,X],axis=1) #concatenated data

#Rename the first column
yX = yX.rename(columns={0: 'TARGET'})

In [0]:
yX.head(2)

**CREATE A CORRELATION MATRIX**

In [0]:
f, ax = plt.subplots(figsize = (15,10))
sns.heatmap(yX.corr().abs(), square=True)

#### **Addressing Multicollinearity**
Multicollinearity occurs when independent variables in a model are correlated. 

In [0]:
# correlation in the df dataset
corrmatrix = df.corr().abs()
corrmatrix = corrmatrix.stack()
corrmatrix[(corrmatrix > 0.6) & (corrmatrix != 0.1).sort_values(ascending = True)]

In [0]:
# correlation in the yx dataset
corrmatrix = yX.corr().abs()
corrmatrix = corrmatrix.stack()
corrmatrix[(corrmatrix > 0.6) & (corrmatrix != 0.1).sort_values(ascending = True)]

Note the high correlation between features that are directly related e.g mean radius, mean perimeter, mean area

We are going to build our model over several algorithms:

- Logistic Regression

- Random Forest

- Support Vector Machine

- Gradient Boosting

- Gaussian Naive Bayes

### **Setup an automatic workflow with pipeline**


The use of pipeline will ensure...

In [0]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

In [0]:
# Logistic Regression
lr_pipeline = Pipeline([('scaler', StandardScaler()), 
                        ('classifier', LR(random_state=5))])

# Random Forest
rf_pipeline = Pipeline([('scaler', StandardScaler()), 
                        ('classifier', RF(random_state=5))                        
])

# Gradient Boosting
gb_pipeline = Pipeline([('scaler', StandardScaler()), 
                        ('classifier', GBC())
                        ])

# Gaussian Naive Bayes
nb_pipeline = Pipeline([('scaler', StandardScaler()), 
                        ('classifier', GNB())])

# SVM
svm_pipeline = Pipeline([('scaler', StandardScaler()), 
                        ('classifier', svm.SVC(gamma=0.01, C=100))])

**Hyperparameter optimization**

Using GridSearch and Cross Validation to optimize on model hyperparameters

In [0]:
from sklearn.model_selection import GridSearchCV

#GridSearch for Logistic Regression
lr_param_range = [0.001, 0.01, 0.1]
lr_class_weight = [{0:0.01, 1:0.99}, {0:0.80, 1:0.20}]
lr_param_grid = [{'classifier__C':lr_param_range,
                  'classifier__class_weight':lr_class_weight
                 }]
gridsearch_lr = GridSearchCV(estimator = lr_pipeline,
                             param_grid = lr_param_grid,
                             n_jobs = -1,
                             cv = 5)

In [0]:
#GridSearch for Random Forest

rf_class_weight = [{0:0.01, 1:0.99}, {0:0.10, 1:0.90}, {0:0.80,1:0.20}]
rf_param_grid = [
                 {'class_weight':rf_class_weight,
                  'max_features':["auto", "sqrt", "log2"],
                  'n_estimators': [20, 50],
                  'min_samples_leaf': [5, 10],
                  'min_samples_split': [100, 200],
                  'criterion': ["entropy","gini"]
                 }
                ]
rf_classifier = RF()

gridsearch_rf = GridSearchCV(
                             rf_classifier,
                             param_grid = rf_param_grid,
                             n_jobs = -1,
                             cv = 5,
                             refit = True
                            )

In [0]:
#GridSearch for Gradient Boosting Classifier
gb_param_grid = [{'classifier__n_estimators': [1000],
                  'classifier__max_features': ["sqrt"],
                  'classifier__min_samples_leaf': [9, 13],
                  'classifier__learning_rate': [0.05, 0.01]
                }]
gridsearch_gb = GridSearchCV(estimator = gb_pipeline,
                             param_grid = gb_param_grid,
                             n_jobs = -1,
                             cv = 2
                            )

In [0]:
#GridSearch for Support Vector Machine

svm_param_range = [0.0001, 0.001, 0.01, 0.1, 1.0, 10.0, 100.0, 1000.0]
svm_param_grid = [{'classifier__C': svm_param_range,
                   'classifier__kernel': ['linear']},
                  {'classifier__C': svm_param_range,
                   'classifier__gamma': svm_param_range,
                   'classifier__kernel': ['rbf']}]
gridsearch_svm = GridSearchCV(estimator = svm_pipeline,
                              param_grid = svm_param_grid,
                              n_jobs = -1,
                              cv = 10)

We need to know the total number of entries so we can take half of them

In [0]:
yX.info(verbose=False)

In [0]:
yX_half = yX[0:280]

Extract the features and target that will be used for hyperparameter optimization

In [0]:
X_gs_train = yX_half.iloc[:,1:]
y_gs_train = yX_half.iloc[:,0]

Why use dill

In [0]:
import dill

Fit the dataset to get the hyperparameters and pickle the best_estimator to be used *later*

In [0]:
#Fitting the dataset to get the hyperparameters
gridsearch_lr.fit(X_gs_train, y_gs_train)
gridsearch_best_estimator_lr = gridsearch_lr.best_estimator_
dill.dump(gridsearch_best_estimator_lr, open('LogisticRegression_gridsearch_classweight.pkl', 'wb'))

In [0]:
gridsearch_rf.fit(X_gs_train, y_gs_train)
gridsearch_best_estimator_rf = gridsearch_rf.best_estimator_
dill.dump(gridsearch_best_estimator_rf, open('RandomForest_gridsearch_classweight.pkl', 'wb'))

In [0]:
gridsearch_gb.fit(X_gs_train, y_gs_train)
gridsearch_best_estimator_gb = gridsearch_gb.best_estimator_
dill.dump(gridsearch_best_estimator_gb, open('GradientBoosting_gridsearch_classweight.pkl', 'wb'))

In [0]:
gridsearch_svm.fit(X_gs_train, y_gs_train)
gridsearch_best_estimator_svm = gridsearch_svm.best_estimator_
dill.dump(gridsearch_best_estimator_svm, open('SupportVectorMachine_gridsearch_classweight.pkl', 'wb'))

MODEL TRAINING AND EVALUATION

With the optimized hyperparameters, we can then go ahead and fit models with cross validation. I will also be doing an evaluation of each model for accuracy, precision, recall, as well as the area under the Receiver Operating Characteristic (ROC) curve

Importing packages for model selection and evaluation

In [0]:
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import auc
from sklearn.metrics import classification_report
from sklearn.metrics import plot_roc_curve
from scipy import interp

Function to draw a confusion matrix

In [0]:
def draw_confusion_matrix(conf_matrix, classifier_name):
    fig, ax = plt.subplots(figsize = (5,5))
    ax.matshow(conf_matrix, cmap = plt.cm.Greens, alpha=0.3)
    for i in range(conf_matrix.shape[0]):
         for j in range(conf_matrix.shape[1]):
                 ax.text(x=j, y=i, s=conf_matrix[i, j], va = 'center', ha = 'center')
    plt.title('Confusion Matrix for %s' % classifier_name)
    plt.xlabel('Predicted label')
    plt.ylabel('True')
    
    plt.tight_layout()
    plt.show()

Cross Validation function

In the run_cv function, I will apply the cross-validation as well as draw the ROC curves for each training-validation fold.
Area Under Curve (AUC) plots are independent of the fraction of each class and are therefore a very good metric for evaluating a classifier performance in the case of when you are dealing with an imbalanced data set such as this one.

In [0]:
def run_cv(X, Y, classifier, clf_name):
    
    #Construct a kfolds object
    kf = StratifiedKFold(n_splits=5,shuffle=True)

    
    accuracy_scores = []
            
    #Initialize ROC variables
    tprs = []
    aucs = []
    mean_fpr = np.linspace(0, 1, 100)
    
    
    clf = classifier
    
    y_pred_full = Y.copy()
    
    fig, ax = plt.subplots()
    
    #Iterate through folds
    for i, (train_index, test_index) in enumerate(kf.split(X,Y)):
        
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        Y_train = Y.iloc[train_index]
        Y_test = Y.iloc[test_index]
            
     #Train the classifier on the training data
        clf_fit = clf.fit(X_train,Y_train)
          
         #Get probabilities and compute area under ROC curve
        viz = plot_roc_curve(clf, X_test, Y_test, name = 'ROC fold {}'.format(i), alpha=0.3, lw = 1, ax = ax)            
        interp_tpr = interp(mean_fpr, viz.fpr, viz.tpr)
        interp_tpr[0] = 0.0
        tprs.append(interp_tpr)
        aucs.append(viz.roc_auc)
        
        #Obtain a prediction on the test set
        y_pred = clf_fit.predict(X_test)
        
        #Map the prediction for this fold to the full dataset
        y_pred_full.iloc[test_index] = y_pred
    
        #Calculate the accuracy of the prediction on current fold
        accuracy_scores.append(accuracy_score(y_true=Y_test, y_pred=y_pred))
        
        
        
    
    #Get Evaluation metrics    
    #Draw ROC Curve    
    mean_tpr = np.mean(tprs, axis = 0)
    mean_tpr[-1] = 1.0
    mean_auc = auc(mean_fpr, mean_tpr)
    std_auc = np.std(aucs)
    
    
    ax.plot([0, 1], 
             [0, 1], 
             '--', 
             color=(0.6, 0.6, 0.6), 
             label='Luck',
             alpha =.8
            )
    
    ax.plot([0, 0, 1], 
             [0, 1, 1], 
             lw=2,
             linestyle=':',
             color='black',
             label='Perfect Performance')
    
    ax.plot(mean_fpr, 
             mean_tpr, 
             'k--',
             label=r'Mean ROC (AUC = %0.2f $\pm$ %0.2f)' % (mean_auc, std_auc), lw=2, alpha=.8)
    
#     std_tpr = np.std(tprs, axis = 0)
#     tprs_upper = np.minimum(mean_tpr + std_tpr, 1)
#     tprs_lower = np.maximum(mean_tpr - std_tpr, 0)
#     ax.fill_between(mean_fpr, tprs_lower, tprs_upper, color = 'grey', alpha =.2, label = r'$\pm$ 1 std. dev.')
    
    plt.xlim([-0.05, 1.05])
    plt.ylim([-0.05, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Receiver Operating Characteristic for %s' % classifier_name)
    plt.legend(loc="lower right")
    #plt.tight_layout()
    plt.show()
    
    #Accuracy score
    mean = np.mean(accuracy_scores)
    std = np.std(accuracy_scores)
    print (clf_name + ':' + '\n' + 'cross-validation accuracy')
    print ("%.2f +/- %.3f" % (mean, std))
    print (classification_report(Y, y_pred_full))
    
    #Confusion Matrix
    conf_matrix = confusion_matrix(y_true=Y, y_pred=y_pred_full)
    draw_confusion_matrix(conf_matrix, clf_name)
    
    return clf_fit

Getting the previously pickled files for each algorithm and running cross validation on each algorithm for comparison

In [0]:
#LR
with open('LogisticRegression_gridsearch_classweight.pkl', 'rb') as f:
    LogisticRegression_classifier = dill.load(f)
classifier_name = 'Logistic Regression Classifier'
model_pipeline_lr = run_cv(X_train, y_train, LogisticRegression_classifier, classifier_name)


In [0]:
# Gradient Boosting Classifier
with open('GradientBoosting_gridsearch_classweight.pkl', 'rb') as f:
    GradientBoosting_classifier = dill.load(f)
classifier_name = 'Gradient Boosting'
model_pipeline_gb = run_cv(X_train, y_train, GradientBoosting_classifier, classifier_name)
dill.dump(model_pipeline_gb, open('GradientBoosting_model_AllFeatures.pkl', 'wb'))

In [0]:
#Random Forest Classifier

with open('RandomForest_gridsearch_classweight.pkl', 'rb') as f:
    RandomForest_classifier = dill.load(f)
classifier_name = 'Random Forest'
model_pipeline_rf = run_cv(X_train, y_train, RandomForest_classifier, classifier_name)
dill.dump(model_pipeline_rf,open('RandomForest_model_AllFeatures.pkl', 'wb'))

In [0]:
#Gaussian Naive Bayes

classifier_name = 'Gaussian Naive Bayes'
GaussianNB_classifier = GNB()
model_pipeline_gnb = run_cv(X_train, y_train, GaussianNB_classifier, classifier_name)

In [0]:
with open('SupportVectorMachine_gridsearch_classweight.pkl', 'rb') as f:
    SupportVectorMachine_classifier = dill.load(f)
classifier_name = 'Support Vector Machine'
model_pipeline_svm = run_cv(X_train, y_train, SupportVectorMachine_classifier, classifier_name)